In [1]:
# Ελεγχος version pyspark .. Θέλουμε το 2.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [2]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [4]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("TemperatureAnalysis").getOrCreate()

In [7]:
agn = "data/agn.us.txt"
ainv = "data/ainv.us.txt"
ale = "data/ale.us.txt"

agn_data = spark.read.csv(agn, header=True)
ainv_data = spark.read.csv(ainv, header=True)
ale_data = spark.read.csv(ale, header=True)

In [8]:
# Ελεγχος αν διαβαστηκαν σωστα τα αρχεια
print(agn_data.show())

+----------+------+------+------+------+-------+-------+
|      Date|  Open|  High|   Low| Close| Volume|OpenInt|
+----------+------+------+------+------+-------+-------+
|2005-01-03| 32.31| 32.31|31.527|31.616|1027044|      0|
|2005-01-04|31.527|31.616| 31.22|31.338|1927762|      0|
|2005-01-05|30.971|31.051|30.714|30.843| 943399|      0|
|2005-01-06|30.843|31.398|30.764| 31.26| 662398|      0|
|2005-01-07| 31.26| 31.26|30.456|30.566|1087886|      0|
|2005-01-10|30.506| 31.23|30.486|30.863| 896381|      0|
|2005-01-11|30.843|30.903|30.199|30.377| 835841|      0|
|2005-01-12|30.456|30.466|29.872| 30.07|1476041|      0|
|2005-01-13| 30.07|30.605|29.882|30.129| 772578|      0|
|2005-01-14|30.189|30.337| 30.07|30.239| 453538|      0|
|2005-01-18| 30.09|30.724|29.862|30.496| 592979|      0|
|2005-01-19|30.506|30.605|29.852| 30.08| 694685|      0|
|2005-01-20| 30.06|30.407| 30.05|  30.1| 858644|      0|
|2005-01-21|30.129|30.307|29.584|29.723|1040463|      0|
|2005-01-24|29.763|29.931|29.28

In [ ]:
# Ελεγχος αν διαβαστηκαν σωστα τα αρχεια
print(ainv_data.show())

In [ ]:
# Ελεγχος αν διαβαστηκαν σωστα τα αρχεια
print(ale_data.show())